In [2]:
import asyncio
import os
from concurrent.futures import ThreadPoolExecutor

import polars as pl
from queue import Queue
import threading
import asyncio
import websockets
import json
import time


async def binance_price_ws(queue, symbols, throttle=0.3):
    streams = "/".join(f"{s}@ticker" for s in symbols)
    url = f"wss://fstream.binance.com/stream?streams={streams}"

    async with websockets.connect(url) as ws:
        print("Connected to Binance Futures Ticker WebSocket!")
        last_update = 0
        while True:
            msg = await ws.recv()
            now = time.time()
            if now - last_update >= throttle:
                last_update = now
                data = json.loads(msg)
                stream = data.get('stream')
                await queue.put(
                    (stream, data['data'])
                )


async def transform_queue_message(queue):
    while True:
        data = await queue.get()
        print(data)


async def main():
    symbols = ["btcusdt", "ethusdt", "bnbusdt"]
    queue = asyncio.Queue()


    await asyncio.gather(
        binance_price_ws(queue, symbols),
        transform_queue_message(queue),
    )


await main()

Connected to Binance Futures Ticker WebSocket!
('btcusdt@ticker', {'e': '24hrTicker', 'E': 1767737717424, 's': 'BTCUSDT', 'p': '-985.10', 'P': '-1.046', 'w': '93088.13', 'c': '93157.80', 'Q': '0.002', 'o': '94142.90', 'h': '94415.40', 'l': '91209.00', 'v': '185073.314', 'q': '17228129063.60', 'O': 1767651300000, 'C': 1767737717420, 'F': 7076558588, 'L': 7080135706, 'n': 3568482})
('bnbusdt@ticker', {'e': '24hrTicker', 'E': 1767737717757, 's': 'BNBUSDT', 'p': '1.200', 'P': '0.131', 'w': '910.453', 'c': '914.690', 'Q': '0.14', 'o': '913.490', 'h': '924.840', 'l': '892.030', 'v': '654988.85', 'q': '596336381.447', 'O': 1767651300000, 'C': 1767737717755, 'F': 1994757872, 'L': 1995810738, 'n': 1049992})
('ethusdt@ticker', {'e': '24hrTicker', 'E': 1767737718080, 's': 'ETHUSDT', 'p': '41.31', 'P': '1.277', 'w': '3248.43', 'c': '3277.27', 'Q': '0.061', 'o': '3235.96', 'h': '3308.00', 'l': '3180.71', 'v': '4522187.257', 'q': '14689997737.44', 'O': 1767651300000, 'C': 1767737718077, 'F': 7170782

CancelledError: 

# Computation Heavy Pipeline

In [2]:
import asyncio
import os
import polars as pl
import threading
import asyncio
import websockets
import json
import time


def heavy_calc(message):
    open = message.get("o")
    high = message.get("h")
    low = message.get("l")
    close = message.get("c")

    return open * high *low + close


async def binance_price_ws(queue, symbols, throttle=0.3):
    streams = "/".join(f"{s}@ticker" for s in symbols)
    url = f"wss://fstream.binance.com/stream?streams={streams}"

    async with websockets.connect(url) as ws:
        print("Connected to Binance Futures Ticker WebSocket!")
        last_update = 0
        while True:
            msg = await ws.recv()
            now = time.time()
            if now - last_update >= throttle:
                last_update = now
                data = json.loads(msg)
                stream = data.get('stream')
                await queue.put(
                    (stream, data['data'])
                )


def transform_queue_message(queue, executor):
    loop = asyncio.get_event_loop()
    while True:
        data = queue.get()
        result = loop.run_in_executor(executor, heavy_calc, data)
        print(result)


async def main():
    symbols = ["btcusdt", "ethusdt", "bnbusdt"]
    import queue
    queue = queue.Queue()
    from concurrent.futures import ProcessPoolExecutor
    executor = ProcessPoolExecutor(max_workers=10)

    await asyncio.gather(
        binance_price_ws(queue, symbols),
        transform_queue_message(queue, executor),
    )



asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop